# TensorFlow
- Eine Open-Source-Plattform für maschinelles Lernen. 
- Entwicklung von künstlichen neuronalen Netzwerken
- Unterstützt mehrere Programmiersprachen u. a. Python
- Dokumentation: https://www.tensorflow.org/
- TensorFlow Playground - zum Lernen und Ausprobieren: https://playground.tensorflow.org/
- Muss installiert werden https://www.tensorflow.org/install


## Keras
- neben TensorFlow auch eine Open Source Deep-Learning-Bibliothek, geschrieben in Python.
- Dokumentation:https://keras.io/
- bietet eine einheitliche Schnittstelle für verschiedene Backends, darunter TensorFlow

### Installation
Wir installieren `TensorFlow` mit pip. Dabei wird auch `Keras` automatisch mitinstalliert.

In [1]:
# !pip install tensorflow

In [2]:
!pip show tensorflow

Name: tensorflow
Version: 2.18.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: C:\Users\AndreasWagner\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: tensorflow-intel
Required-by: 


In [3]:
!pip show keras

Name: keras
Version: 3.8.0
Summary: Multi-backend Keras
Home-page: 
Author: 
Author-email: Keras team <keras-users@googlegroups.com>
License: Apache License 2.0
Location: C:\Users\AndreasWagner\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: absl-py, h5py, ml-dtypes, namex, numpy, optree, packaging, rich
Required-by: tensorflow_intel


### Eine Regressionsaufgabe mit KNN
In diesem Beispiel wollen wir das bekannte Beispiel von Hauspreis-Modell mit Hilfe eines KNNs bearbeiten.
Dazu gehen wir folgende Schritte:
- Daten einlesen und vorbereiten
- Ein KNN-Modell erzeugen
- Schichten hinzufügen
- Modell kompilieren
- Modell trainieren
- Modell testen

## Daten einlesen und vorbereiten

Daten einlesen und Dataframe erzeugen.

In [4]:
import pandas as pd
df = pd.read_csv('house_data.csv')
df.head()

,sq_feet,num_bedrooms,num_bathrooms,sale_price
0,785,2,2,170461
1,1477,2,2,271651
2,712,1,1,139912
3,3233,3,3,603246
4,1581,2,1,278603


Daten normalisieren: KNN-Modelle benötigen skalierte Daten, damit alle Merkmale gleichmäßig behandelt werden. 

In [5]:
from sklearn.preprocessing import MinMaxScaler
feature_set=df.drop('sale_price', axis='columns') 
feature_scaler=MinMaxScaler()
features_scaled=feature_scaler.fit_transform(feature_set)
features_scaled[:5] # die ersten 5 Elemente anzeigen

array([[0.08145184, 0.33333333, 0.5       ],
       [0.27922264, 0.33333333, 0.5       ],
       [0.06058874, 0.        , 0.        ],
       [0.78108031, 0.66666667, 1.        ],
       [0.30894541, 0.33333333, 0.        ]])

In [6]:
from sklearn.preprocessing import MinMaxScaler
target_scaler=MinMaxScaler()
target=df[['sale_price']]
target_scaled=target_scaler.fit_transform(target)
target_scaled[:5] # die ersten 5 Objekte

array([[0.10939552],
       [0.26461204],
       [0.06253605],
       [0.7732495 ],
       [0.2752758 ]])

In [7]:
feature_names=feature_set.columns.values
feature_names

array(['sq_feet', 'num_bedrooms', 'num_bathrooms'], dtype=object)

In [8]:
# aus skalierten Daten ein neues Dataframe bilden
df_scaled=pd.DataFrame(features_scaled,columns=feature_names)
df_scaled['sale_price']=target_scaled
df_scaled.head()

,sq_feet,num_bedrooms,num_bathrooms,sale_price
0,0.081452,0.333333,0.5,0.109396
1,0.279223,0.333333,0.5,0.264612
2,0.060589,0.000000,0.0,0.062536
3,0.781080,0.666667,1.0,0.773249
4,0.308945,0.333333,0.0,0.275276


Daten aufsplitten - Train und Test Daten

In [9]:
from sklearn.model_selection import train_test_split
X=df_scaled.drop('sale_price',axis='columns')
y=df_scaled['sale_price']
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.25)

## Ein KNN-Modell erzeugen

#### _Achtung_
`Keras` kann direkt oder über `TensorFlow` importieret werden.

In [10]:
# Über TensorFlow 

# from tensorflow.keras.layers import Dense
# from tensorflow.keras.models import Sequential

# Direkt
from keras.layers import Dense
from keras.models import Sequential

`Dense`: ist die einfachste und am meisten verwendete Form einer KNN-Schicht. Die Neuronen einer Dense-Schicht sind eng mit Neuronen der vorherigen Schicht verknüpft und bekommen über sie Signale vermittelt. Für eine ausführliche Erklärung siehe [hier](https://analyticsindiamag.com/a-complete-understanding-of-dense-layers-in-neural-networks/)

`Sequential`: Sequentielle Algorithmen in Machine Learning gehen davon aus, dass die Angaben einer Schicht von Angaben der vorigen Schicht abhängig sind.

In [11]:
model = Sequential() # Model erstellen

Wir addieren die erste Dense-Schicht zu unserem Modell

In [12]:
model.add(Dense(50, input_dim=3, activation='relu')) # 50: Anzahl der Knoten, input_dim=Anzahl der Features, activation=Aktivierungsfunktion

c:\Users\AndreasWagner\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


und dazu kommen noch weitere Schichten

In [13]:
model.add(Dense(100, activation='relu')) # eine weitere versteckte Schicht (hidden layer) mit 100 Knoten
model.add(Dense(50, activation='relu')) # eine weitere versteckte Schicht (hidden layer) mit 50 Knoten

und schließlich kommt die Ausgabeschicht (output)

In [14]:
model.add(Dense(1, activation='linear')) # 1 Konten, Weil Ausgabe nur eine Zahl (Preis)

Insgesamt haben wir jetzt 5 Schichten: input, dann eine Schicht mit 50 Knoten, dann eine mit 100, dann eine mit 50 dann Output.

Im nächsten Schritt soll das Modell noch kompiliert werden:

In [15]:
model.compile(
loss='mean_squared_error', # Loss Function - Das Ziel ist, diese zu minimieren
optimizer='SGD') # stochastic gradient descent - Optimierungsfunktion - wie die Loss Function minimieren

Das Modell trainieren
_Hinweis_ Das Training von KNN-Modellen dauert in der Regel länger im Vergleich zu einfachen Modellen aus SKLearn.

In [16]:
model.fit(X_train,y_train,epochs=100,batch_size=8,shuffle=True,verbose=2)

Epoch 1/100
938/938 - 2s - 2ms/step - loss: 0.0133
Epoch 2/100
938/938 - 1s - 1ms/step - loss: 5.6377e-04
Epoch 3/100
938/938 - 1s - 1ms/step - loss: 4.1337e-04
Epoch 4/100
938/938 - 1s - 1ms/step - loss: 3.7367e-04
Epoch 5/100
938/938 - 1s - 1ms/step - loss: 3.5803e-04
Epoch 6/100
938/938 - 1s - 1ms/step - loss: 3.4833e-04
Epoch 7/100
938/938 - 1s - 1ms/step - loss: 3.4211e-04
Epoch 8/100
938/938 - 1s - 1ms/step - loss: 3.3862e-04
Epoch 9/100
938/938 - 1s - 1ms/step - loss: 3.3461e-04
Epoch 10/100
938/938 - 1s - 1ms/step - loss: 3.3211e-04
Epoch 11/100
938/938 - 1s - 1ms/step - loss: 3.2980e-04
Epoch 12/100
938/938 - 1s - 1ms/step - loss: 3.2801e-04
Epoch 13/100
938/938 - 1s - 1ms/step - loss: 3.2751e-04
Epoch 14/100
938/938 - 1s - 1ms/step - loss: 3.2571e-04
Epoch 15/100
938/938 - 1s - 1ms/step - loss: 3.2404e-04
Epoch 16/100
938/938 - 1s - 1ms/step - loss: 3.2384e-04
Epoch 17/100
938/938 - 1s - 1ms/step - loss: 3.2296e-04
Epoch 18/100
938/938 - 1s - 1ms/step - loss: 3.2280e-04
Epoch

Das trainierte Modell testen

In [17]:
from sklearn.metrics import mean_absolute_error as mae
print('Mittlerer absoluter Fehler (MAE) in Traindaten',mae(y_train,model.predict(X_train)))
print('Mittlerer absoluter Fehler (MAE) in Testdaten',mae(y_test,model.predict(X_test)))

235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Mittlerer absoluter Fehler (MAE) in Traindaten 0.013818898312221469
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Mittlerer absoluter Fehler (MAE) in Testdaten 0.014504929055354616


Das Sequential-Modell zeigt eine wesentlich bessere Leistung im Vergleich zu unserem bisherigen LinearRegression Modell.  
_Hinweis_ Es ist aber nicht immer so!